In [1]:
from __future__ import print_function
import os
import numpy as np
from sklearn.preprocessing import normalize
import cv2
import matplotlib.pyplot as plt

import imageio
from multiprocessing.pool import Pool
import shutil
import pickle
import os

from functools import partial

In [2]:
def get_best_sgbm():
    window_size = 5
    return cv2.StereoSGBM_create(
           minDisparity= -0, #Because our images are rectified.
           numDisparities = 128,
           blockSize = 7,
           P1 = 8 * 3 * window_size ** 2,
           P2 = 40 * 3 * window_size ** 2,
           disp12MaxDiff=1,
           uniquenessRatio=21,
           speckleWindowSize = 100,
           speckleRange=1,
           preFilterCap=73,
           mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
        )

def get_wls_filtered(left_matcher, imgL, left_disp_wls, imgR):
    right_matcher = cv2.ximgproc.createRightMatcher(left_matcher)
    wls_filter = cv2.ximgproc.createDisparityWLSFilter(left_matcher)
    wls_filter.setLambda(80000)
    wls_filter.setSigmaColor(1.3)
    right_disp_wls = right_matcher.compute(imgR, imgL).astype(np.float32)
    # right_disp_wls = cv2.normalize(src=right_disp_wls, dst=right_disp_wls, beta=0, alpha=255, norm_type=cv2.NORM_MINMAX).astype(np.float32)
    # left_disp_wls = cv2.normalize(src=left_disp_wls, dst=left_disp_wls, beta=0, alpha=255, norm_type=cv2.NORM_MINMAX).astype(np.float32)
    return wls_filter.filter(left_disp_wls, imgL, None, right_disp_wls)

def get_masked_map(map, threshold):
    mask = map > threshold
    return map[mask].reshape(map.shape)

def flag_poor_matches(map):

    map[map < 0] = -2 # So now everything that isn't matched is -2
    return np.nan_to_num(map, True, -1, -1, -1)


In [3]:
def create_and_dump(data_dir, folderpath, fileidx, scenename, left_matcher, block_matcher, numerator, out_dir):
    imgL = cv2.imread(os.path.join(data_dir, folderpath, '0', 'left_rgb', f'{fileidx}.jpg'), 0)
    imgR = cv2.imread(os.path.join(data_dir, folderpath, '0', 'right_rgb', f'{fileidx}.jpg'), 0)
    
    # SGBM output
    left_disp_sgbm = left_matcher.compute(imgL,imgR).astype(np.float32)/ 16.0
    left_depth_sgbm = numerator/left_disp_sgbm
    left_depth_sgbm = flag_poor_matches(left_depth_sgbm)
    left_depth_sgbm = (100 * (left_depth_sgbm + 2)).astype(np.uint16)  # Add offset and multiply by 100 for uint16
    sgbm_dir = os.path.join(out_dir, 'sgbm', folderpath, '0', 'stereo_depth')
    os.makedirs(sgbm_dir, exist_ok=True)
#     np.save(os.path.join(sgbm_dir, f"{fileidx}.npy"), (100 * left_depth_sgbm).astype(np.int16)) # m to cm
    cv2.imwrite(os.path.join(sgbm_dir, f"{fileidx}.png"), left_depth_sgbm) # in cm

    # BM output
    left_disp_bm = block_matcher.compute(imgL, imgR).astype(np.float32)/ 16.0
    left_depth_bm = numerator/left_disp_bm
    left_depth_bm = flag_poor_matches(left_depth_bm)    
    left_depth_bm = (100 * (left_depth_bm + 2)).astype(np.uint16)  # Add offset and multiply by 100 for uint16
    bm_dir = os.path.join(out_dir, 'bm', folderpath, '0', 'stereo_depth')
    os.makedirs(bm_dir, exist_ok=True)
#     np.save(os.path.join(bm_dir, f"{fileidx}.npy"), (100 * left_depth_bm).astype(np.int16)) # m to cm
    cv2.imwrite(os.path.join(bm_dir, f"{fileidx}.png"), left_depth_bm) # in cm

In [4]:
def write_depth(out_dir, filepaths):
    left_matcher = get_best_sgbm()

    block_matcher = cv2.StereoBM_create(numDisparities = 64, blockSize = 15)

    f = 320
    b = 0.2
    numerator = (f * b)

    for filepath in filepaths:
        data_dir, folderpath, fileidx = filepath.split()
        scenename = os.path.basename(folderpath)        
        create_and_dump(data_dir, folderpath, fileidx, scenename, left_matcher, block_matcher, numerator, out_dir)

In [5]:
data_dir = '/scratch/shantanu.singh/habitat_data'
split = 'HabitatVal'

filepaths = []
for scene in os.listdir(os.path.join(data_dir, split)):
    scene_files = ['{} {}/{} {}'.format(data_dir, split, scene, os.path.splitext(filename)[0]) for 
                   filename in os.listdir(os.path.join(data_dir, split, scene, '0', 'pose'))]
    filepaths.extend(scene_files)
    
len(filepaths)

17212

In [6]:
nprocs = 18
chunk_size = len(filepaths)//nprocs + 1
chunks = [filepaths[i*chunk_size:(i+1)*chunk_size] for i in range(nprocs)]

out_dir = '/scratch/shantanu.singh/stereo_depth'
pool_fn = partial(write_depth, (out_dir))

with Pool(processes=nprocs) as P:
    P.map(pool_fn, chunks)

/home/shantanu.singh/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in true_divide
/home/shantanu.singh/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/home/shantanu.singh/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/home/shantanu.singh/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in true_divide
/home/shantanu.singh/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in true_divide
/home/shantanu.singh/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/home/shantanu.singh/miniconda3/envs/py3

In [10]:
# Test if npy and png are same upon reading.

# tdir = '/scratch/shantanu.singh/stereo_depth/HabitatVal/TbHJrupSAjP/0'

# d1 = (cv2.imread(os.path.join(tdir, 'sgbm_depth2', '0.png'), -1)).astype(np.float32)/100 - 2
# d2 = np.load(os.path.join(tdir, 'sgbm_depth', '0.npy')).astype(np.float32)/100

# np.sum(np.isclose(d1, d2)) * 100 / (d1.size)

100.0

In [5]:
def compute_errors(gt, pred, mask=None):
    """Computation of error metrics between predicted and ground truth depths
    """
    if mask is None:
        mask = np.ones_like(gt) == 1
        
    valid = np.sum(mask)/mask.size
        
    gt = gt[mask]
    pred = pred[mask]
        
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25     ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()

    rmse = (gt - pred) ** 2
    rmse = np.sqrt(rmse.mean())

    rmse_log = (np.log(gt) - np.log(pred)) ** 2
    rmse_log = np.sqrt(rmse_log.mean())

    abs_rel = np.mean(np.abs(gt - pred) / gt)

    sq_rel = np.mean(((gt - pred) ** 2) / gt)

    return abs_rel, sq_rel, rmse, rmse_log, a1, a2, a3, valid

In [6]:
def evaluate_depth(filepaths, out_dir, pred_dir):
    scores = []
    for filepath in filepaths:
        data_dir, folderpath, fileidx = filepath.split()
        scenename = os.path.basename(folderpath)
        
        depth_path = os.path.join(data_dir, folderpath, '0', 'left_depth', f'{fileidx}.png')
        gt = cv2.imread(depth_path, -1).astype(np.float32)/6553.5
        gt = np.clip(gt, 0.1, 10)
        
        depth_path = os.path.join(out_dir, folderpath, '0', pred_dir, f'{fileidx}.png')
        pred = cv2.imread(depth_path, -1).astype(np.float32)/100 - 2
        pred = np.clip(pred, 0.1, 10)
        
        mask = pred > 0
        
        scores.append(compute_errors(gt, pred, mask))
    
    count = len(scores)
    score_sum = np.sum(np.array(scores), axis=0)
    return score_sum, count

In [7]:
data_dir = '/scratch/shantanu.singh/habitat_data'
split = 'HabitatTrain'
pred_dir = 'sgbm_depth'

nprocs = 18

overall_scores = dict()
for scene in os.listdir(os.path.join(data_dir, split)):
    filepaths = ['{} {}/{} {}'.format(data_dir, split, scene, os.path.splitext(filename)[0]) for 
                   filename in os.listdir(os.path.join(data_dir, split, scene, '0', 'pose'))]
    
    print(scene, len(filepaths))

    chunk_size = len(filepaths)//nprocs + 1
    chunks = [filepaths[i*chunk_size:(i+1)*chunk_size] for i in range(nprocs)]

    out_dir = '/scratch/shantanu.singh/stereo_depth'
    pool_fn = partial(evaluate_depth, out_dir=out_dir, pred_dir=pred_dir)

    with Pool(processes=nprocs) as P:
        scene_scores = P.map(pool_fn, chunks)
        
    total_score = np.empty(shape=(8,))
    total_count = 0
    for score, count in scene_scores:
        total_score += score
        total_count += count
    
    overall_scores[scene] = total_score/total_count

EDJbREhghzL 760
dhjEzFoUFzH 12692
aayBHfsNo7d 1184
29hnd4uzFmX 815
pRbA3pwrgk9 212
gTV8FGcVJC9 876
5LpN3gDmAk7 142
mJXqzFtmKg4 3481
7y3sRwLe3Va 1138
D7G3Y4RVNrH 269
uNb9QFRL6hY 1461
gZ6f7yhEvPG 132
e9zR4mvMWw7 1174
D7N2EKCX4Sj 1614
Uxmj2M2itWa 1204
qoiz87JEwZ2 1374
ULsKaCPVFJR 1656
1LXtFkjw3qL 1788
rPc6DW4iMge 1551
JmbYfDe2QKZ 811
82sE5b5pLXE 3503
VVfe2KiqLaN 904
VLzqgDo317F 277
JeFG25nYj2p 3330
8WUmhLawc2A 1765
vyrNrziPKCB 12418
Pm6F8kyY3z2 377
ZMojNkEp431 1454
1pXnuDYAj8r 1998
PX4nDJXEHrG 3642
b8cTxDM8gDG 1109
XcA2TqTSSAj 177
SN83YJsR3w2 3598
GdvgFV5R1Z5 219
s8pcmisQ38h 265
Vvot9Ly1tCj 7013
YmJkqBEsHnH 662
jh4fc5c5qoQ 722
kEZ7cmS4wCh 5955
E9uDoFAP3SH 6160
VFuaQ6m2Qom 4292
759xd9YjKW5 2147
cV4RVeZvu5T 248
B6ByNegPMKs 35252
S9hNv5qa7GM 1291
ac26ZMwG7aT 4644
VzqfbhrpDEA 5813
JF19kD82Mey 433
2n8kARJN3HM 2564
ur6pFq6Qu1A 13225
HxpKQynjfin 274
5q7pvUzZiYa 1002
PuKPg4mmafe 5014
17DRP5sb8fy 732
r47D5H71a5s 1734
sKLMLpTHeUy 1700
V2XKFyX4ASd 790
i5noydFURQK 635
sT4fr6TAbpF 3213
p5wJjkQkbXX 482

In [9]:
overall_scores

{'EDJbREhghzL': array([0.53213735, 0.97557162, 0.87185928, 1.32730939, 0.59875215,
        0.62010631, 0.640012  , 1.        ]),
 'dhjEzFoUFzH': array([ 3.85787221, 34.10571562,  3.25864023,  1.85469478,  0.55753752,
         0.57461749,  0.59375715,  1.00338796]),
 'aayBHfsNo7d': array([ 4.72147492, 30.51760031,  3.71781915,  2.88841679,  0.94192745,
         0.96156144,  0.97992279,  1.59628378]),
 '29hnd4uzFmX': array([0.54694229, 1.68783713, 1.3694614 , 1.69858092, 0.66277716,
        0.67088237, 0.67882825, 1.0809816 ]),
 'pRbA3pwrgk9': array([0.58782518, 1.21375353, 1.35695121, 1.9095925 , 0.76697226,
        0.77821807, 0.7843752 , 1.21698113]),
 'gTV8FGcVJC9': array([0.55653227, 2.3260324 , 1.4107832 , 1.46175283, 0.66484726,
        0.67633273, 0.69050721, 1.01369863]),
 '5LpN3gDmAk7': array([ 5.94736045, 21.92206038,  2.5739635 ,  2.48013493,  0.84981012,
         0.85875919,  0.86400702,  1.34507042]),
 'mJXqzFtmKg4': array([1.42567442, 7.5083335 , 1.41336072, 1.59517079, 0.

In [8]:
pickle.dump(overall_scores, open('/scratch/shantanu.singh/stereo_depth/scores_habitat_train.pkl', 'wb'))